In [1]:
# filepath: src/model.py
import pytesseract
import cv2
#to start the virtual environment, run: venv\Scripts\activate 
# Set tesseract path if not in PATH
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def extract_text_from_invoice(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found at path: {image_path}")
    text = pytesseract.image_to_string(image)
    return text

print(extract_text_from_invoice("data/invoice_sample.png"))


invoice

FROM INVOICE # us-001
East Repair Inc. INVOICE DATE 1102/2019
1912 Harvest Lane P.O.# 2312/2019
New York, NY 12210
DUE DATE 26/02/2019
BILL TO SHIP TO
John Smith John Smith
2 Court Square 3787 Pineview Drive
New York, NY 12210 ‘Cambridge, MA 12210
QTY DESCRIPTION UNIT PRICE AMOUNT
1 Front and rear brake cables 100.00 100.00
2 New set of pedal arms 15.00 30.00
3 Labor 3hrs, 5.00 1.00
Subtotal 145.00
Sales Tax 6.25% 9.06

TOTAL $154.06

TERMS & CONDITIONS

Payment is due within 15 days

Please make checks payable to: East Repair Inc.

Swath

